In [1]:
import warnings

warnings.filterwarnings('ignore')

from megnet.models import MEGNetModel


2024-04-22 06:29:09.868019: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:29:09.868054: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Load formation energy model

In [2]:
model_form = MEGNetModel.from_file('../mvl_models/mp-2018.6.1/formation_energy.hdf5')

2024-04-22 06:29:15.236958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:29:15.237028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:29:15.237077: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 06:29:15.237123: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Cou

### Get the embedding layer

In [3]:
embedding_layer = [i for i in model_form.layers if i.name.startswith('embedding')][0]
embedding = embedding_layer.get_weights()[0]
print('Embedding matrix dimension is ', embedding.shape)

Embedding matrix dimension is  (95, 16)


The embedding matrix size is 95 x 16, so that the maximum atomic number (94) in the MP database can find the corresponding row in the embedding matrix. 

### Construct a new model and set embeddings

In [6]:
model = MEGNetModel(100, 2, nvocal=95, embedding_dim=16)

In [7]:
# find the embedding layer  index in all the model layers
embedding_layer_index = [i for i, j in enumerate(model.layers) if j.name.startswith('atom_embedding')][0]

# Set the weights to our previous embedding
model.layers[embedding_layer_index].set_weights([embedding])

# Freeze the weights
model.layers[embedding_layer_index].trainable = False

Now that `model` should have the same embeddings as the pre-trained model, and the weights won't change during training.